In [1]:
#! install python3
"""
BBOCapstoneWeeklyInputGeneratorAndAnalysis.ipynb

End-to-end weekly workflow for the BBO capstone challenge, with Gaussian Process surrogate modelling and acquisition-based query generation.
-------------------------------------------------------------------------------------------------------------------------------------------
What it does:
1) Loads the provided historical seed datasets (points_*/values_*).
2) Appends all weekly observations you have so far (Week 1 .. Week K).
3) Fits one Gaussian Process (GP) surrogate per function.
4) Automatically selects the best kernel by maximising log-marginal likelihood (LML)
   across a small kernel pool: Constant*(RBF | Matern 1.5 | Matern 2.5) + WhiteKernel.
5) Chooses the next query point using an acquisition function (EI / PI / UCB).
6) Tunes exploration vs exploitation per-function:
   - EI/PI uses ξ (xi): smaller = more exploit, larger = more explore
   - UCB uses β (beta): smaller = more exploit, larger = more explore
7) Saves:
   - Portal-formatted next-week queries
   - Diagnostics (chosen kernel, LML, ξ/β, μ/σ at x*, exploration proxy)
   - Plots to ./plots/funcXX/
   - A history snapshot to ./history/week_XX.json (inputs + outputs so far + generated next)

Optional (Module 16 alignment):
- Fits a small Neural Network surrogate (sklearn MLPRegressor) per function and logs:
  - Validation score (quick sanity check)
  - Finite-difference "gradient" around the best predicted region (directional signal)
  This mirrors deep-learning concepts (feature learning / gradients) without replacing the GP,
  because GP uncertainty is still the most useful signal for exploration.

How to use (step-by-step):
1) Put this file in your repo root (same folder as README.md).
2) Ensure numpy, scipy, scikit-learn, matplotlib are installed.
3) Update the WEEKLY_INPUTS_ALL / WEEKLY_OUTPUTS_ALL lists with your latest week.
4) Run:  python bbo_gp_weekly_generator_updated.py
5) Copy the printed portal lines (Week K+1 queries) into the submission portal.
6) After results return, paste Week K+1 outputs into WEEKLY_OUTPUTS_ALL and rerun.

Domain:
- Assumes each dimension is in [0, 1] and the portal requires values that begin with "0".
- This script clips to [0.0, 0.999999] before formatting.

Objective:
- Maximisation for all 8 functions.
"""

from __future__ import annotations

import json
import os
from dataclasses import dataclass
from datetime import datetime, timezone
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple

import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import norm

from sklearn.decomposition import PCA
from sklearn.exceptions import ConvergenceWarning
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, Matern, RBF, WhiteKernel
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import warnings

# Silence noisy GP optimisation warnings (length_scales hitting bounds, etc.).
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# =========================
# CONFIG
# =========================

ACQUISITION = "ei"  # "ei", "pi", "ucb"
N_CANDIDATES = 70000
RNG_SEED = 123
DECIMALS = 6

# GP settings
NOISE_ALPHA = 1e-10
RESTARTS_LOW_D = 10   # 2D–3D
RESTARTS_MID_D = 6    # 4D–5D
RESTARTS_HIGH_D = 2   # 6D–8D

# Kernel bounds (increase if you keep hitting upper bounds)
LENGTH_SCALE_BOUNDS = (1e-3, 50.0)
WHITE_NOISE_BOUNDS = (1e-12, 1e-1)

# Explore vs exploit decision rule:
# If last week's y is within tol_frac of best-so-far (relative to range), exploit; else explore.
EXPLOIT_TOL_FRAC_OF_RANGE = 0.10

# EI/PI exploration parameter
XI_EXPLOIT = 0.001
XI_EXPLORE = 0.05

# UCB exploration parameter
BETA_EXPLOIT = 1.0
BETA_EXPLORE = 3.0

# Output folders
PLOTS_DIR = Path("plots")
HISTORY_DIR = Path("history")

# Plot params
GRID_RES_2D = 120
MAX_POINTS_FOR_PCA_SCATTER = 5000

# Optional: Neural Network surrogate diagnostics (Module 16 alignment)
USE_NN_SURROGATE_DIAGNOSTICS = True
NN_HIDDEN_LAYER_SIZES = (64, 64)
NN_MAX_ITER = 2000
NN_RANDOM_STATE = 7


# =========================
# PORTAL FORMATTER
# =========================
def fmt_query(x: np.ndarray, decimals: int = DECIMALS) -> str:
    """
    Format x as '0.xxxxxx-0.yyyyyy-...' with six decimals.
    The portal requires each component to begin with 0, so we clip to [0.0, 0.999999].
    """
    x = np.asarray(x, float).reshape(-1)
    x = np.clip(x, 0.0, 0.999999)
    return "-".join([f"{v:.{decimals}f}" for v in x])


# =========================
# ACQUISITION FUNCTIONS
# =========================
def acq_ucb(mu: np.ndarray, sigma: np.ndarray, beta: float) -> np.ndarray:
    return mu + beta * sigma


def acq_pi(mu: np.ndarray, sigma: np.ndarray, y_best: float, xi: float) -> np.ndarray:
    sigma = np.maximum(sigma, 1e-12)
    z = (mu - y_best - xi) / sigma
    return norm.cdf(z)


def acq_ei(mu: np.ndarray, sigma: np.ndarray, y_best: float, xi: float) -> np.ndarray:
    sigma = np.maximum(sigma, 1e-12)
    imp = mu - y_best - xi
    z = imp / sigma
    return imp * norm.cdf(z) + sigma * norm.pdf(z)


# =========================
# EXPLORE / EXPLOIT TUNING
# =========================
def decide_mode_maximise(y_last: float, y_hist: np.ndarray, tol_frac: float) -> str:
    """
    Decide 'exploit' vs 'explore' using last week's output vs best-so-far (relative to range).
    """
    y_hist = np.asarray(y_hist, float).reshape(-1)
    best = float(np.max(y_hist))
    worst = float(np.min(y_hist))
    span = max(best - worst, 1e-12)
    dist_frac = (best - float(y_last)) / span
    return "exploit" if dist_frac <= tol_frac else "explore"


def tune_params(mode: str, acquisition: str) -> Dict[str, float]:
    acquisition = acquisition.lower().strip()
    if acquisition in ("ei", "pi"):
        return {"xi": XI_EXPLOIT if mode == "exploit" else XI_EXPLORE}
    if acquisition == "ucb":
        return {"beta": BETA_EXPLOIT if mode == "exploit" else BETA_EXPLORE}
    raise ValueError("ACQUISITION must be one of: ei, pi, ucb")


# =========================
# KERNEL SELECTION BY LML
# =========================
def restarts_for_dim(dim: int) -> int:
    if dim <= 3:
        return RESTARTS_LOW_D
    if dim <= 5:
        return RESTARTS_MID_D
    return RESTARTS_HIGH_D


def make_kernel_pool(dim: int) -> List[Any]:
    ls0 = 0.30 if dim <= 3 else (0.40 if dim <= 5 else 0.60)
    base = [
        RBF(length_scale=np.ones(dim) * ls0, length_scale_bounds=LENGTH_SCALE_BOUNDS),
        Matern(length_scale=np.ones(dim) * ls0, length_scale_bounds=LENGTH_SCALE_BOUNDS, nu=1.5),
        Matern(length_scale=np.ones(dim) * ls0, length_scale_bounds=LENGTH_SCALE_BOUNDS, nu=2.5),
    ]
    pool: List[Any] = []
    for bk in base:
        pool.append(
            ConstantKernel(1.0, (1e-3, 1e3)) * bk
            + WhiteKernel(noise_level=1e-5, noise_level_bounds=WHITE_NOISE_BOUNDS)
        )
    return pool


def fit_gp_and_lml(X: np.ndarray, y: np.ndarray, kernel: Any, seed: int, n_restarts: int):
    gp = GaussianProcessRegressor(
        kernel=kernel,
        alpha=NOISE_ALPHA,
        normalize_y=True,
        n_restarts_optimizer=n_restarts,
        random_state=seed,
    )
    gp.fit(X, y)
    lml = float(gp.log_marginal_likelihood(gp.kernel_.theta))
    return gp, lml


def fit_best_gp_by_lml(X: np.ndarray, y: np.ndarray, dim: int, seed: int):
    pool = make_kernel_pool(dim)
    n_restarts = restarts_for_dim(dim)

    best_gp: Optional[GaussianProcessRegressor] = None
    best_lml = -np.inf
    details: List[Tuple[str, float]] = []

    for j, kernel in enumerate(pool):
        gp, lml = fit_gp_and_lml(X, y, kernel, seed=seed + 17 * j, n_restarts=n_restarts)
        kstr = str(gp.kernel_)
        details.append((kstr, float(lml)))
        if lml > best_lml:
            best_lml = float(lml)
            best_gp = gp

    assert best_gp is not None
    return best_gp, str(best_gp.kernel_), float(best_lml), details


# =========================
# NN SURROGATE DIAGNOSTICS (optional)
# =========================
def nn_surrogate_diagnostics(X: np.ndarray, y: np.ndarray, *, seed: int) -> Dict[str, Any]:
    """
    Train a small MLP surrogate (tabular NN) and report:
    - holdout R^2 (if enough data)
    - finite-difference gradient around the best predicted point
    This is intentionally lightweight and used for diagnostics, not for uncertainty.
    """
    X = np.asarray(X, float)
    y = np.asarray(y, float).reshape(-1)

    report: Dict[str, Any] = {"enabled": True}

    if X.shape[0] < 6:
        report["note"] = "Too few points for reliable NN diagnostics."
        return report

    # Holdout split (tiny datasets => keep it simple)
    test_size = 0.25 if X.shape[0] >= 12 else 0.33
    Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=test_size, random_state=seed)

    model = Pipeline(
        steps=[
            ("scaler", StandardScaler()),
            ("mlp", MLPRegressor(
                hidden_layer_sizes=NN_HIDDEN_LAYER_SIZES,
                activation="relu",
                solver="adam",
                alpha=1e-4,
                learning_rate_init=1e-3,
                max_iter=NN_MAX_ITER,
                random_state=seed,
            )),
        ]
    )
    model.fit(Xtr, ytr)
    r2 = float(model.score(Xte, yte))
    report["holdout_r2"] = r2

    # Find a local "best" input according to the NN and approximate gradient there
    rng = np.random.default_rng(seed + 999)
    Xcand = rng.uniform(0.0, 1.0, size=(5000, X.shape[1]))
    ypred = model.predict(Xcand)
    x0 = Xcand[int(np.argmax(ypred))]

    eps = 1e-3
    grad = np.zeros_like(x0)
    base = float(model.predict(x0.reshape(1, -1))[0])
    for d in range(x0.size):
        x_plus = x0.copy()
        x_minus = x0.copy()
        x_plus[d] = min(0.999999, x_plus[d] + eps)
        x_minus[d] = max(0.0, x_minus[d] - eps)
        f_plus = float(model.predict(x_plus.reshape(1, -1))[0])
        f_minus = float(model.predict(x_minus.reshape(1, -1))[0])
        grad[d] = (f_plus - f_minus) / (2 * eps)

    report["nn_best_x"] = x0.tolist()
    report["nn_best_pred"] = float(np.max(ypred))
    report["fd_grad_at_nn_best"] = grad.tolist()
    report["fd_grad_l1"] = float(np.sum(np.abs(grad)))
    report["fd_grad_top_dims"] = np.argsort(-np.abs(grad))[: min(5, x0.size)].tolist()
    return report


# =========================
# EXPLORATION PROXY
# =========================
def exploration_ratio(mu: float, sigma: float, acquisition: str, xi: float, beta: float) -> float:
    """
    A simple proxy: how much uncertainty contributes relative to mean at the chosen point.
    """
    acquisition = acquisition.lower().strip()
    if acquisition == "ucb":
        return float((beta * sigma) / (abs(mu) + beta * sigma + 1e-12))
    return float(sigma / (abs(mu) + sigma + xi + 1e-12))


# =========================
# PROPOSE NEXT POINT
# =========================
def propose_next_point(
    X: np.ndarray,
    y: np.ndarray,
    dim: int,
    *,
    acquisition: str,
    xi: float,
    beta: float,
    seed: int,
    n_candidates: int,
):
    rng = np.random.default_rng(seed)

    gp, kernel_str, best_lml, lml_details = fit_best_gp_by_lml(X, y, dim=dim, seed=seed)

    # Candidate set: uniform in [0,1]^d
    Xcand = rng.uniform(0.0, 1.0, size=(n_candidates, dim))
    mu, sigma = gp.predict(Xcand, return_std=True)
    mu = mu.reshape(-1)
    sigma = sigma.reshape(-1)

    y_best = float(np.max(y))
    a = acquisition.lower().strip()

    if a == "ei":
        score = acq_ei(mu, sigma, y_best=y_best, xi=xi)
    elif a == "pi":
        score = acq_pi(mu, sigma, y_best=y_best, xi=xi)
    elif a == "ucb":
        score = acq_ucb(mu, sigma, beta=beta)
    else:
        raise ValueError("ACQUISITION must be one of: ei, pi, ucb")

    best_idx = int(np.argmax(score))
    x_next = Xcand[best_idx]
    mu_best = float(mu[best_idx])
    sigma_best = float(sigma[best_idx])
    ratio = exploration_ratio(mu_best, sigma_best, acquisition, xi, beta)

    report = {
        "kernel": kernel_str,
        "best_lml": float(best_lml),
        "lml_candidates": [(k, float(l)) for k, l in lml_details],
        "mu_at_choice": mu_best,
        "sigma_at_choice": sigma_best,
        "exploration_ratio": ratio,
    }
    return x_next, fmt_query(x_next), report, gp


# =========================
# PLOTTING
# =========================
def ensure_func_plot_dir(func_idx: int) -> Path:
    d = PLOTS_DIR / f"func{func_idx:02d}"
    d.mkdir(parents=True, exist_ok=True)
    return d


def plot_y_over_iterations(func_idx: int, key: str, y: np.ndarray, out_path: Path) -> None:
    y = np.asarray(y, float).reshape(-1)
    it = np.arange(1, len(y) + 1)
    plt.figure()
    plt.plot(it, y, marker="o")
    plt.axhline(np.max(y), linestyle="--")
    plt.title(f"{key} (Function {func_idx}) - y over iterations")
    plt.xlabel("Observation index")
    plt.ylabel("y")
    plt.tight_layout()
    plt.savefig(out_path, dpi=150)
    plt.close()


def plot_2d_gp_mean_contour(
    func_idx: int,
    key: str,
    gp: GaussianProcessRegressor,
    X_obs: np.ndarray,
    y_obs: np.ndarray,
    x_next: np.ndarray,
    out_path: Path,
    grid_res: int = GRID_RES_2D,
) -> None:
    xs = np.linspace(0.0, 1.0, grid_res)
    ys = np.linspace(0.0, 1.0, grid_res)
    X1, X2 = np.meshgrid(xs, ys)
    grid = np.column_stack([X1.ravel(), X2.ravel()])
    mu = gp.predict(grid).reshape(grid_res, grid_res)

    plt.figure()
    plt.contourf(X1, X2, mu, levels=20)
    plt.scatter(X_obs[:, 0], X_obs[:, 1], c=y_obs, marker="o")
    plt.scatter([x_next[0]], [x_next[1]], marker="X", s=120)
    plt.title(f"{key} (Function {func_idx}) - GP mean (2D)")
    plt.xlabel("x1")
    plt.ylabel("x2")
    plt.tight_layout()
    plt.savefig(out_path, dpi=150)
    plt.close()


def plot_pca_scatter(
    func_idx: int,
    key: str,
    X_obs: np.ndarray,
    y_obs: np.ndarray,
    x_next: np.ndarray,
    out_path: Path,
) -> None:
    X_obs = np.asarray(X_obs, float)
    y_obs = np.asarray(y_obs, float).reshape(-1)

    if X_obs.shape[0] > MAX_POINTS_FOR_PCA_SCATTER:
        idx = np.random.default_rng(RNG_SEED + func_idx).choice(
            X_obs.shape[0], size=MAX_POINTS_FOR_PCA_SCATTER, replace=False
        )
        X_plot = X_obs[idx]
        y_plot = y_obs[idx]
    else:
        X_plot, y_plot = X_obs, y_obs

    X_stack = np.vstack([X_plot, x_next.reshape(1, -1)])
    pca = PCA(n_components=2, random_state=RNG_SEED + func_idx)
    Z = pca.fit_transform(X_stack)
    Z_obs = Z[:-1]
    Z_next = Z[-1]

    plt.figure()
    plt.scatter(Z_obs[:, 0], Z_obs[:, 1], c=y_plot, marker="o")
    plt.scatter([Z_next[0]], [Z_next[1]], marker="X", s=120)
    plt.title(f"{key} (Function {func_idx}) - PCA projection")
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.tight_layout()
    plt.savefig(out_path, dpi=150)
    plt.close()


# =========================
# DATA WRAPPER
# =========================
@dataclass
class FunctionDataset:
    key: str
    dim: int
    X: np.ndarray
    y: np.ndarray

    def __post_init__(self) -> None:
        self.X = np.asarray(self.X, float).reshape(-1, self.dim)
        self.y = np.asarray(self.y, float).reshape(-1)
        if self.X.shape[0] != self.y.shape[0]:
            raise ValueError(f"{self.key}: X rows != y rows")

    def append(self, x_new: np.ndarray, y_new: float) -> None:
        x_new = np.asarray(x_new, float).reshape(1, self.dim)
        self.X = np.vstack([self.X, x_new])
        self.y = np.concatenate([self.y, [float(y_new)]])


# =========================
# HISTORICAL SEED DATA
# =========================
# NOTE: These are your provided seed datasets (unchanged).

# 2D DATASET A (tiny values)
points_2d_a = np.array([
    [0.31940389, 0.76295937],
    [0.57432921, 0.87989810],
    [0.73102363, 0.73299988],
    [0.84035342, 0.26473161],
    [0.65011406, 0.68152635],
    [0.41043714, 0.14755430],
    [0.31269116, 0.07872278],
    [0.68341817, 0.86105746],
    [0.08250725, 0.40348751],
])
values_2d_a = np.array([
    1.3226770395454077e-79,
    1.0330782375230975e-46,
    7.710875114502849e-16,
    3.341771007676023e-124,
    -0.0036060626443634764,
    -2.1592490357331095e-54,
    -2.0890932702320842e-91,
    2.5350011535584046e-40,
    3.6067711901420254e-81,
])

# 2D DATASET B
points_2d_b = np.array([
    [0.66579958, 0.12396913],
    [0.87779099, 0.77862750],
    [0.14269907, 0.34900513],
    [0.84527543, 0.71112027],
    [0.45464714, 0.29045518],
    [0.57771284, 0.77197318],
    [0.43816606, 0.68501826],
    [0.34174959, 0.02869772],
    [0.33864816, 0.21386725],
])
values_2d_b = np.array([
    0.5389961189269181,
    0.42058623962798264,
    -0.06562362443733738,
    0.293992912410866,
    0.2149645101004509,
    0.023105549798190586,
    0.24461934400448035,
    0.0387490151561584,
    -0.013857618149729824,
])

# 3D DATASET
points_3d = np.array([
    [0.17152521, 0.34391687, 0.24873720],
    [0.24211446, 0.64407427, 0.27243281],
    [0.53490572, 0.39850092, 0.17338873],
    [0.49258141, 0.61159319, 0.34017639],
    [0.13462167, 0.21991724, 0.45820622],
    [0.34552327, 0.94135983, 0.26936348],
    [0.15183663, 0.43999062, 0.99088187],
    [0.64550284, 0.39714294, 0.91977134],
    [0.74691195, 0.28419631, 0.22629985],
    [0.17047699, 0.69703240, 0.14916943],
    [0.22054934, 0.29782524, 0.34355534],
    [0.66601366, 0.67198515, 0.24629530],
    [0.04680895, 0.23136024, 0.77061759],
    [0.60009728, 0.72513573, 0.06608864],
])
values_3d = np.array([
    -0.11212220046256897,
    -0.08796286022736445,
    -0.11141465429532400,
    -0.034835313350078584,
    -0.04800758439218157,
    -0.11062091307282658,
    -0.39892551314630110,
    -0.11386851478863991,
    -0.13146060864136055,
    -0.09418956091057398,
    -0.04694740582651916,
    -0.10596503573558178,
    -0.11804825644688696,
    -0.036377828071632486,
])

# 4D DATASET A (negative)
points_4d_a = np.array([
    [0.89698105, 0.72562797, 0.17540431, 0.70169437],
    [0.88935640, 0.49958786, 0.53926886, 0.50878344],
    [0.25094624, 0.03369313, 0.14538002, 0.49493242],
    [0.34696206, 0.00625040, 0.76056361, 0.61302356],
    [0.12487118, 0.12977019, 0.38440048, 0.28707610],
    [0.80130271, 0.50023109, 0.70664456, 0.19510284],
    [0.24770826, 0.06044543, 0.04218635, 0.44132425],
    [0.74670224, 0.75709150, 0.36935306, 0.20656628],
    [0.40066503, 0.07257425, 0.88676825, 0.24384229],
    [0.62607060, 0.58675126, 0.43880578, 0.77885769],
    [0.95713529, 0.59764438, 0.76611385, 0.77620991],
    [0.73281243, 0.14524998, 0.47681272, 0.13336573],
    [0.65511548, 0.07239183, 0.68715175, 0.08151656],
    [0.21973443, 0.83203134, 0.48286416, 0.08256923],
    [0.48859419, 0.21196510, 0.93917791, 0.37619173],
    [0.16713049, 0.87655456, 0.21723954, 0.95980098],
    [0.21691119, 0.16608583, 0.24137226, 0.77006248],
    [0.38748784, 0.80453226, 0.75179548, 0.72382744],
    [0.98562189, 0.66693268, 0.15678328, 0.85653480],
    [0.03782483, 0.66485335, 0.16198218, 0.25392378],
    [0.68348638, 0.90277010, 0.33541983, 0.99948256],
    [0.17034731, 0.75695908, 0.27652049, 0.53123150],
    [0.85965692, 0.91959232, 0.20613873, 0.09779683],
    [0.28213837, 0.50598691, 0.53053084, 0.09630162],
    [0.32607578, 0.47236690, 0.45319200, 0.10588734],
    [0.94838936, 0.89451301, 0.85163782, 0.55219629],
    [0.66495539, 0.04656628, 0.11677747, 0.79371778],
    [0.57776561, 0.42877174, 0.42582587, 0.24900741],
    [0.73861301, 0.48210263, 0.70936644, 0.50397001],
])
values_4d_a = np.array([
    -22.108287785435817,
    -14.601396631953161,
    -11.699932463834426,
    -16.053765110568296,
    -10.069633426012889,
    -15.487082537755288,
    -12.681684976815650,
    -16.026399768983520,
    -17.049234647535723,
    -12.741765988316185,
    -27.316396356142608,
    -13.527648872277506,
    -16.679115197479913,
    -16.507158564282340,
    -17.817999338829490,
    -26.561820829779602,
    -12.758324216781741,
    -19.441557624451722,
    -28.903273673833770,
    -13.702746938161251,
    -29.427091401989347,
    -11.565741989583191,
    -26.857786437773637,
    -7.9667753510303925,
    -6.702089254839066,
    -32.625660215962455,
    -19.989497926795560,
    -4.025542281908162,
    -13.122782331852594,
])

# 4D DATASET B (positive)
points_4d_b = np.array([
    [0.19144708, 0.03819337, 0.60741781, 0.41458414],
    [0.75865295, 0.53651774, 0.65600038, 0.36034155],
    [0.43834987, 0.80433970, 0.21024527, 0.15129482],
    [0.70605083, 0.53419196, 0.26424335, 0.48208755],
    [0.83647799, 0.19360965, 0.66389270, 0.78564888],
    [0.68343225, 0.11866264, 0.82904591, 0.56757661],
    [0.55362148, 0.66734998, 0.32380582, 0.81486975],
    [0.35235627, 0.32224153, 0.11697937, 0.47311252],
    [0.15378571, 0.72938169, 0.42259844, 0.44307417],
    [0.46344227, 0.63002451, 0.10790646, 0.95764390],
    [0.67749115, 0.35850951, 0.47959222, 0.07288048],
    [0.58397341, 0.14724265, 0.34809746, 0.42861465],
    [0.30688872, 0.31687813, 0.62263448, 0.09539906],
    [0.51114177, 0.81795700, 0.72871042, 0.11235362],
    [0.43893338, 0.77409176, 0.37816709, 0.93369621],
    [0.22418902, 0.84648049, 0.87948418, 0.87851568],
    [0.72526172, 0.47987049, 0.08894684, 0.75976022],
    [0.35548161, 0.63961937, 0.41761768, 0.12260384],
    [0.11987923, 0.86254031, 0.64333133, 0.84980383],
])
values_4d_b = np.array([
    64.443439863301,
    18.30137959857266,
    0.1129397953712203,
    4.210898128938665,
    258.3705254462536,
    78.43438888779464,
    57.57153693261287,
    109.5718755614928,
    8.847991759070865,
    233.22361017104996,
    24.423088313942344,
    64.42014681963983,
    63.47671578508436,
    79.72912992694343,
    355.8068177560159,
    1088.8596181962705,
    28.866751637393822,
    45.181570346703786,
    431.6127567592104,
])

# 5D DATASET (used for Function 6 in your mapping)
points_5d = np.array([
    [0.72818610, 0.15469257, 0.73255167, 0.69399651, 0.05640131],
    [0.24238435, 0.84409997, 0.57780910, 0.67902128, 0.50195289],
    [0.72952261, 0.74810620, 0.67977464, 0.35655228, 0.67105368],
    [0.77062024, 0.11440374, 0.04677993, 0.64832428, 0.27354905],
    [0.61881230, 0.33180214, 0.18728787, 0.75623847, 0.32883480],
    [0.78495809, 0.91068235, 0.70812010, 0.95922543, 0.00491150],
    [0.14511079, 0.89668460, 0.89632223, 0.72627154, 0.23627199],
    [0.94506907, 0.28845905, 0.97880576, 0.96165559, 0.59801594],
    [0.12572016, 0.86272469, 0.02854433, 0.24660527, 0.75120624],
    [0.75759436, 0.35583141, 0.01652290, 0.43420720, 0.11243304],
    [0.53679690, 0.30878091, 0.41187929, 0.38822518, 0.52252830],
    [0.95773967, 0.23566857, 0.09914585, 0.15680593, 0.07131737],
    [0.62930790, 0.80348368, 0.81140844, 0.04561319, 0.11062446],
    [0.02173531, 0.42808424, 0.83593944, 0.48948866, 0.51108173],
    [0.43934426, 0.69892383, 0.42682022, 0.10947609, 0.87788847],
    [0.25890557, 0.79367771, 0.64211390, 0.19667346, 0.59310318],
    [0.43216593, 0.71561781, 0.34181910, 0.70499988, 0.61496184],
    [0.78287982, 0.53633586, 0.44328356, 0.85969983, 0.01032599],
    [0.92177620, 0.93187122, 0.41487637, 0.59505727, 0.73562569],
])
values_5d = np.array([
    -0.7142649478202404,
    -1.2099552446764819,
    -1.6721999396105658,
    -1.5360577085694833,
    -0.8292365522578722,
    -1.2470489267904978,
    -1.2337863805718483,
    -1.6943434420704928,
    -2.5711696316081234,
    -1.3091163528236960,
    -1.1447848512705794,
    -1.9126771431925136,
    -1.6228389517771730,
    -1.3566821093823407,
    -2.0184253993747820,
    -1.7025578405650035,
    -1.2942469649550403,
    -0.9357565553342914,
    -2.1557677641786004,
])

# 6D DATASET (Function 7)
points_6d = np.array([
    [0.27262382, 0.32449536, 0.89710881, 0.83295115, 0.15406269, 0.79586362],
    [0.54300258, 0.92469390, 0.34156746, 0.64648585, 0.71844033, 0.34313266],
    [0.09083225, 0.66152938, 0.06593091, 0.25857701, 0.96345285, 0.64026540],
    [0.11886697, 0.61505494, 0.90581639, 0.85530030, 0.41363143, 0.58523563],
    [0.63021764, 0.83809690, 0.68001305, 0.73189509, 0.52673671, 0.34842921],
    [0.76491917, 0.25588292, 0.60908422, 0.21807904, 0.32294277, 0.09579366],
    [0.05789554, 0.49167222, 0.24742222, 0.21811844, 0.42042833, 0.73096984],
    [0.19525188, 0.07922665, 0.55458046, 0.17056682, 0.01494418, 0.10703171],
    [0.64230298, 0.83687455, 0.02179269, 0.10148801, 0.68307083, 0.69241640],
])
values_6d = np.array([
    0.6044326958745716,
    0.5627530668655433,
    0.007503236678049401,
    0.06142430250355775,
    0.27304680126759867,
    0.08374657232015563,
    1.3649683044991994,
    0.09264495494793601,
    0.017869598722495817,
])

# 8D DATASET (Function 8)
points_8d = np.array([
    [0.60499445,0.29221502,0.90845275,0.35550624,0.20166872,0.57533801,0.31031095,0.73428138],
    [0.17800696,0.56622265,0.99486184,0.21032501,0.32015266,0.70790879,0.63538449,0.10713163],
    [0.00907698,0.81162615,0.52052036,0.07568668,0.26511183,0.09165169,0.59241515,0.36732026],
    [0.50602816,0.65373012,0.36341078,0.17798105,0.09372830,0.19742533,0.75582690,0.29247234],
    [0.35990926,0.24907568,0.49599717,0.70921498,0.11498719,0.28920692,0.55729515,0.59388173],
    [0.77881834,0.00341950,0.33798313,0.51952778,0.82090699,0.53724669,0.55134710,0.66003209],
    [0.90864932,0.06224970,0.23825955,0.76660355,0.13233596,0.99024381,0.68806782,0.74249594],
    [0.58637144,0.88073573,0.74502075,0.54603485,0.00964888,0.74899176,0.23090707,0.09791562],
    [0.76113733,0.85467239,0.38212433,0.33735198,0.68970832,0.30985305,0.63137968,0.04195607],
    [0.98493320,0.69950626,0.99888550,0.18014846,0.58014315,0.23108719,0.49082694,0.31368272],
    [0.11207131,0.43773566,0.59659878,0.59277563,0.22698177,0.41010452,0.92123758,0.67475276],
    [0.79188751,0.57619134,0.69452836,0.28342378,0.13675546,0.27916186,0.84276726,0.62532792],
    [0.14355030,0.93741452,0.23232482,0.00904349,0.41457893,0.40932517,0.55377852,0.20584080],
    [0.76991655,0.45875909,0.55900044,0.69460444,0.50319902,0.72834638,0.78425353,0.66313109],
    [0.05644741,0.06595555,0.02292868,0.03878647,0.40393544,0.80105533,0.48830701,0.89308498],
    [0.86243745,0.48273382,0.28186940,0.54410223,0.88749026,0.38265469,0.60190199,0.47646169],
    [0.35151190,0.59006494,0.90943630,0.67840835,0.21282566,0.08846038,0.41015300,0.19572429],
    [0.73590364,0.03461189,0.72803027,0.14742652,0.29574314,0.44511731,0.97517969,0.37433978],
    [0.68029397,0.25510465,0.86218799,0.13439582,0.32632920,0.28790687,0.43501048,0.36420013],
    [0.04432925,0.01358149,0.25819824,0.57764416,0.05127992,0.15856307,0.59103012,0.07795293],
    [0.77834548,0.75114565,0.31414221,0.90298577,0.33538166,0.38632267,0.74897249,0.98875510],
    [0.89888711,0.52364170,0.87678325,0.21869645,0.90026089,0.28276624,0.91107791,0.47239822],
    [0.14512029,0.11932754,0.42088822,0.38760861,0.15542283,0.87517163,0.51055967,0.72861058],
    [0.33895442,0.56693202,0.37675110,0.09891573,0.65945169,0.24554809,0.76248278,0.73215347],
    [0.17615002,0.29396143,0.97567997,0.79393631,0.92340076,0.03084229,0.80325452,0.59589758],
    [0.02894663,0.02827906,0.48137155,0.61317460,0.67266045,0.02211341,0.60148330,0.52488505],
    [0.19263987,0.63067728,0.41679584,0.49052929,0.79608602,0.65456706,0.27624119,0.29551759],
    [0.94318502,0.21885062,0.72118408,0.42459707,0.98690200,0.53518298,0.71474318,0.96009372],
    [0.53272140,0.83369260,0.07139900,0.11681148,0.73069311,0.93737559,0.86650798,0.12790200],
    [0.44709584,0.84395253,0.72954612,0.63915138,0.40928714,0.13264569,0.03590888,0.44683847],
    [0.38222497,0.55713584,0.85310163,0.33379569,0.26572127,0.48087292,0.23764706,0.76863196],
    [0.53281953,0.86230848,0.53826712,0.04944293,0.71970119,0.90670590,0.10823094,0.52534791],
    [0.39486519,0.33180167,0.74075430,0.69786172,0.73740444,0.78377681,0.25449546,0.87114551],
    [0.98594539,0.87305363,0.07039262,0.05358729,0.73415296,0.52025852,0.81104004,0.10336036],
    [0.96457339,0.97397979,0.66375335,0.66221599,0.67312167,0.90523762,0.45887462,0.56091750],
    [0.47207071,0.16820264,0.08642757,0.45265551,0.48061922,0.62243949,0.92897446,0.11253627],
    [0.85600695,0.63889370,0.32619202,0.66850311,0.24029837,0.21029889,0.16754636,0.96358986],
    [0.81003174,0.63504604,0.26954758,0.86960534,0.66192159,0.25225873,0.76567003,0.89054867],
    [0.79625252,0.00703653,0.35569738,0.48756605,0.74051962,0.70665010,0.99291449,0.38173437],
])
values_8d = np.array([
    7.398721101163708,
    7.005227361900734,
    8.45948161622808,
    8.284007811285548,
    8.606116791392116,
    8.541747923679363,
    7.327434575740623,
    7.299872046566419,
    7.957874742347002,
    5.5921933895401965,
    7.854540990501387,
    6.791985783133633,
    8.976554022457023,
    7.3790829035972365,
    9.598482002566342,
    8.159983191736115,
    7.13162396619294,
    6.767962534878629,
    7.433744072022712,
    9.013075145673822,
    7.310893815253165,
    5.841067313187262,
    9.141639493309754,
    8.817558441363609,
    6.451943125106222,
    8.83074504574746,
    9.34427428080805,
    6.887846394035938,
    8.042212541982503,
    7.692368045766445,
    7.923758772464366,
    8.42175923792568,
    8.27806239964175,
    7.113457163948368,
    6.402588414582601,
    8.472936316651243,
    7.977684585372973,
    7.460872194740431,
    7.436593526746213,
])


# =========================
# WEEKLY OBSERVATIONS (Week 1..4) - UPDATE THIS EACH WEEK
# =========================
# IMPORTANT:
# - Each week inputs is a list of 8 np.arrays (one per function)
# - Each week outputs is a list/array length 8
# - Append new weeks at the end

week1_inputs = [
    np.array([0.372451, 0.684219]),
    np.array([0.815624, 0.243907]),
    np.array([0.214578, 0.739215, 0.563482]),
    np.array([0.691245, 0.182734, 0.534817, 0.408263]),
    np.array([0.456182, 0.827364, 0.319547, 0.672415]),
    np.array([0.238415, 0.564738, 0.792164, 0.413826, 0.689541]),
    np.array([0.127394, 0.458216, 0.839275, 0.364182, 0.592736, 0.715204]),
    np.array([0.091284, 0.376452, 0.648219, 0.823615, 0.214739, 0.587341, 0.732916, 0.459128]),
]
week1_outputs = [
    np.float64(1.5748453958061012e-48),
    np.float64(0.18039410845450976),
    np.float64(-0.03855585192956583),
    np.float64(-8.961029494112594),
    np.float64(49.85674062082695),
    np.float64(-1.5130754270634885),
    np.float64(0.4692517257441858),
    np.float64(7.9402805027841),
]

week2_inputs = [
    np.array([0.263232, 0.692120]),
    np.array([0.999999, 0.310918]),
    np.array([0.171156, 0.820335, 0.517707]),
    np.array([0.757035, 0.095945, 0.402324, 0.338791]),
    np.array([0.310629, 0.834492, 0.396231, 0.799219]),
    np.array([0.203018, 0.547341, 0.773500, 0.466517, 0.605858]),
    np.array([0.000000, 0.578647, 0.913483, 0.367752, 0.528534, 0.777934]),
    np.array([0.094269, 0.433556, 0.540599, 0.814786, 0.194082, 0.529488, 0.725916, 0.380393]),
]
week2_outputs = [
    np.float64(-8.412448709899239e-70),
    np.float64(0.10235357020556336),
    np.float64(-0.03061042303613871),
    np.float64(-12.209283145814855),
    np.float64(151.27449679067803),
    np.float64(-1.3242461523900468),
    np.float64(0.3975173339469103),
    np.float64(8.2293794088801),
]

week3_inputs = [
    np.array([0.704820, 0.153340]),
    np.array([0.696501, 0.036321]),
    np.array([0.969648, 0.771320, 0.854665]),
    np.array([0.416966, 0.404427, 0.356931, 0.406136]),
    np.array([0.513167, 0.831782, 0.996220, 0.999037]),
    np.array([0.370837, 0.173596, 0.693576, 0.962889, 0.168614]),
    np.array([0.225757, 0.439856, 0.295544, 0.723542, 0.196561, 0.797924]),
    np.array([0.063471, 0.108669, 0.036989, 0.129464, 0.927252, 0.469168, 0.101991, 0.123106]),
]
week3_outputs = [
    np.float64(-1.594263249992658e-107),
    np.float64(0.4377146055554574),
    np.float64(-0.036486297266169324),
    np.float64(0.6588288043220065),
    np.float64(2967.924604693576),
    np.float64(-0.6016622426617185),
    np.float64(0.25444565237832933),
    np.float64(9.9182466822364),
]

# Week 4 (portal confirmed)
week4_inputs = [
    np.array([0.763424, 0.919401]),
    np.array([0.000020, 0.608134]),
    np.array([0.114597, 0.538378, 0.000022]),
    np.array([0.425041, 0.428923, 0.330132, 0.432393]),
    np.array([0.207234, 0.049908, 0.993246, 0.999556]),
    np.array([0.486971, 0.421311, 0.700489, 0.897495, 0.304443]),
    np.array([0.109513, 0.742227, 0.296665, 0.334240, 0.160402, 0.722658]),
    np.array([0.151714, 0.013381, 0.259014, 0.150928, 0.965973, 0.652543, 0.242138, 0.674780]),
]
week4_outputs = [
    np.float64(2.8055863313478976e-72),
    np.float64(-0.09297034681235537),
    np.float64(-0.10392147861855096),
    np.float64(0.1659692061914524),
    np.float64(1562.3727909790937),
    np.float64(-0.4440705103815761),
    np.float64(0.5817504892733333),
    np.float64(9.8848985633335),
]

WEEKLY_INPUTS_ALL = [week1_inputs, week2_inputs, week3_inputs, week4_inputs]
WEEKLY_OUTPUTS_ALL = [week1_outputs, week2_outputs, week3_outputs, week4_outputs]


# =========================
# HISTORY SNAPSHOT
# =========================
def save_week_snapshot(
    *,
    week_k: int,
    next_week_inputs: List[np.ndarray],
    next_week_portal_lines: List[str],
    diagnostics: List[Dict[str, Any]],
) -> Path:
    """
    Save a JSON snapshot to ./history/week_XX.json containing:
    - all observed weeks so far (inputs + outputs)
    - next-week generated queries
    - diagnostics per function
    """
    HISTORY_DIR.mkdir(parents=True, exist_ok=True)

    payload: Dict[str, Any] = {
        "generated_at": datetime.now(timezone.utc).isoformat(),
        "acquisition": ACQUISITION,
        "week_k_observed": week_k,
        "weekly_inputs": [
            [np.asarray(x).tolist() for x in week_inputs] for week_inputs in WEEKLY_INPUTS_ALL
        ],
        "weekly_outputs": [
            [float(v) for v in week_outputs] for week_outputs in WEEKLY_OUTPUTS_ALL
        ],
        "next_week_inputs": [x.tolist() for x in next_week_inputs],
        "next_week_portal_lines": next_week_portal_lines,
        "diagnostics": diagnostics,
    }

    out_path = HISTORY_DIR / f"week_{week_k:02d}_to_week_{week_k+1:02d}.json"
    out_path.write_text(json.dumps(payload, indent=2))
    return out_path


# =========================
# MAIN
# =========================
def main() -> None:
    # Validate weekly inputs/outputs structure
    if len(WEEKLY_INPUTS_ALL) != len(WEEKLY_OUTPUTS_ALL):
        raise RuntimeError("WEEKLY_INPUTS_ALL and WEEKLY_OUTPUTS_ALL must have the same length.")
    for w, (wi, wo) in enumerate(zip(WEEKLY_INPUTS_ALL, WEEKLY_OUTPUTS_ALL), start=1):
        if len(wi) != 8:
            raise RuntimeError(f"Week {w} inputs must have 8 entries (one per function).")
        if np.asarray(wo).shape != (8,):
            raise RuntimeError(f"Week {w} outputs must be shape (8,), got {np.asarray(wo).shape}.")

    # Build per-function datasets from historical data
    funcs = [
        FunctionDataset("f1_2d_a", 2, points_2d_a, values_2d_a),
        FunctionDataset("f2_2d_b", 2, points_2d_b, values_2d_b),
        FunctionDataset("f3_3d",   3, points_3d,   values_3d),
        FunctionDataset("f4_4d_a", 4, points_4d_a, values_4d_a),
        FunctionDataset("f5_4d_b", 4, points_4d_b, values_4d_b),
        FunctionDataset("f6_5d",   5, points_5d,   values_5d),
        FunctionDataset("f7_6d",   6, points_6d,   values_6d),
        FunctionDataset("f8_8d",   8, points_8d,   values_8d),
    ]

    # Append Week 1..K
    for w_inputs, w_outputs in zip(WEEKLY_INPUTS_ALL, WEEKLY_OUTPUTS_ALL):
        w_outputs = np.asarray(w_outputs, float).reshape(-1)
        for i, f in enumerate(funcs):
            f.append(w_inputs[i], float(w_outputs[i]))

    week_k = len(WEEKLY_OUTPUTS_ALL)
    last_week_outputs = np.asarray(WEEKLY_OUTPUTS_ALL[-1], float).reshape(-1)

    # Prepare output folders
    PLOTS_DIR.mkdir(parents=True, exist_ok=True)
    HISTORY_DIR.mkdir(parents=True, exist_ok=True)

    # Generate next-week queries
    print(f"=== Next WEEK {week_k + 1} QUERIES (PORTAL FORMAT) ===")

    next_points: List[np.ndarray] = []
    portal_lines: List[str] = []
    diagnostics_payload: List[Dict[str, Any]] = []
    debug_rows = []  # for printing + plotting

    for idx, f in enumerate(funcs, start=1):
        mode = decide_mode_maximise(
            y_last=float(last_week_outputs[idx - 1]),
            y_hist=f.y,
            tol_frac=EXPLOIT_TOL_FRAC_OF_RANGE,
        )
        tuned = tune_params(mode, ACQUISITION)
        xi = float(tuned.get("xi", XI_EXPLORE))
        beta = float(tuned.get("beta", BETA_EXPLORE))

        x_next, portal_line, report, gp = propose_next_point(
            f.X,
            f.y,
            f.dim,
            acquisition=ACQUISITION,
            xi=xi,
            beta=beta,
            seed=RNG_SEED + 31 * idx,
            n_candidates=N_CANDIDATES,
        )

        print(portal_line)
        next_points.append(np.asarray(x_next, float))
        portal_lines.append(portal_line)

        # Optional NN diagnostics (Module 16)
        nn_report = None
        if USE_NN_SURROGATE_DIAGNOSTICS:
            nn_report = nn_surrogate_diagnostics(f.X, f.y, seed=NN_RANDOM_STATE + idx)

        diag = {
            "function_index": idx,
            "function_key": f.key,
            "dim": f.dim,
            "mode": mode,
            "acquisition": ACQUISITION.upper(),
            "xi": xi,
            "beta": beta,
            "kernel": report["kernel"],
            "best_lml": report["best_lml"],
            "mu_at_choice": report["mu_at_choice"],
            "sigma_at_choice": report["sigma_at_choice"],
            "exploration_ratio": report["exploration_ratio"],
            "lml_candidates": report["lml_candidates"],
            "nn_diagnostics": nn_report,
        }
        diagnostics_payload.append(diag)
        debug_rows.append((idx, f, mode, xi, beta, report, gp))

    # Save history snapshot (so the repo accumulates week-by-week progress)
    snapshot_path = save_week_snapshot(
        week_k=week_k,
        next_week_inputs=next_points,
        next_week_portal_lines=portal_lines,
        diagnostics=diagnostics_payload,
    )

    # Print diagnostics (human readable)
    print("\n=== DIAGNOSTICS (kernel / xi-beta / explore-exploit) ===\n")
    for (idx, f, mode, xi, beta, report, _gp) in debug_rows:
        print(
            f"Function {idx} ({f.key}): {mode.upper()} | acquisition={ACQUISITION.upper()} "
            f"| xi={xi:.6f} | beta={beta:.3f}"
        )
        print(f"  chosen_kernel: {report['kernel']}")
        print(f"  best_LML: {report['best_lml']:.3f}")
        print(f"  mu(x*): {report['mu_at_choice']:.6f} | sigma(x*): {report['sigma_at_choice']:.6f}")
        print(f"  exploration_ratio (proxy): {report['exploration_ratio']:.3f}")
        print("  LML candidates:")
        for kstr, lml in report["lml_candidates"]:
            print(f"    - {lml:.3f} :: {kstr}")
        print()

    # Plots (per function folder)
    for (idx, f, _mode, _xi, _beta, _report, gp), x_next in zip(debug_rows, next_points):
        fdir = ensure_func_plot_dir(idx)
        plot_y_over_iterations(idx, f.key, f.y, fdir / "y_over_iterations.png")
        if f.dim == 2:
            plot_2d_gp_mean_contour(idx, f.key, gp, f.X, f.y, x_next, fdir / "gp_mean_contour.png")
        else:
            plot_pca_scatter(idx, f.key, f.X, f.y, x_next, fdir / "pca_scatter.png")

    print(f"\nSaved plots under: {PLOTS_DIR.resolve()}")
    print(f"Saved history snapshot: {snapshot_path.resolve()}")


if __name__ == "__main__":
    main()


c:\Users\prasa\anaconda3\Lib\site-packages\pandas\core\computation\expressions.py:22: UserWarning: Pandas requires version '2.10.2' or newer of 'numexpr' (version '2.10.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


=== Next WEEK 5 QUERIES (PORTAL FORMAT) ===
0.804537-0.537328
0.658368-0.997372
0.801681-0.863251-0.481611
0.377690-0.412490-0.408035-0.432458
0.846177-0.998305-0.997510-0.892575
0.427255-0.500377-0.374352-0.971439-0.241657
0.315573-0.394539-0.256298-0.196140-0.834450-0.752520
0.095926-0.169300-0.093217-0.115986-0.902777-0.374681-0.213161-0.359900

=== DIAGNOSTICS (kernel / xi-beta / explore-exploit) ===

Function 1 (f1_2d_a): EXPLOIT | acquisition=EI | xi=0.001000 | beta=3.000
  chosen_kernel: 1.02**2 * RBF(length_scale=[0.0174, 50]) + WhiteKernel(noise_level=8.6e-12)
  best_LML: -17.459
  mu(x*): -0.000230 | sigma(x*): 0.000969
  exploration_ratio (proxy): 0.441
  LML candidates:
    - -17.459 :: 1.02**2 * RBF(length_scale=[0.0174, 50]) + WhiteKernel(noise_level=8.6e-12)
    - -17.833 :: 1.02**2 * Matern(length_scale=[0.019, 50], nu=1.5) + WhiteKernel(noise_level=3.27e-11)
    - -17.682 :: 1.03**2 * Matern(length_scale=[0.0189, 50], nu=2.5) + WhiteKernel(noise_level=5.64e-10)

Functi